# Stable cell lines expressing DAG1 orthologs viral titers

In [ ]:
# Imports
import os
import warnings
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, ticker as mticker

from scipy.interpolate import splrep, splev

# Plotting colors
tol_muted_adjusted = [
    "#000000",
    "#CC6677", 
    "#1f78b4", 
    "#DDCC77", 
    "#117733", 
    "#882255", 
    "#88CCEE",
    "#44AA99", 
    "#999933", 
    "#AA4499", 
    "#EE7733",
    "#CC3311",
    "#DDDDDD",
]

# Seaborn style settings
sns.set(rc={"figure.dpi":300, "savefig.dpi":300})
sns.set_style("ticks")
sns.set_palette(tol_muted_adjusted)

# Suppress warnings
warnings.simplefilter("ignore")

In [ ]:
# Read data
titer_data = pd.read_excel("data/042123_titer_calculations.xlsx")
# Set output directory
out_dir = "figures/"

In [ ]:
# Re-group dataframe
titer_data = titer_data[["Infection_sample", "Titers_(TU/mL)"]]
titer_data["VEP"] = titer_data["Infection_sample"].str.contains("_LASV_")
titer_data["VEP"] = titer_data["VEP"].map({True: "GPC", False: "VSV-G"})
titer_data["Infection_sample"] = titer_data["Infection_sample"].str.replace("_LASV_", "")
titer_data["Infection_sample"] = titer_data["Infection_sample"].str.replace("_VSVG_", "")
titer_data["Infection_sample"] = titer_data["Infection_sample"].str.replace("rep1", "")
titer_data["Infection_sample"] = titer_data["Infection_sample"].str.replace("rep2", "")
titer_data["Infection_sample"] = titer_data["Infection_sample"].str.replace("rep3", "")

titer_data

In [ ]:
# Plot data
plt.figure(figsize=(2,2))
chart = sns.swarmplot(
    data=titer_data, 
    x="Infection_sample", 
    y="Titers_(TU/mL)", 
    hue="VEP", 
    alpha=0.8,
    edgecolor="black",
    linewidth=0.5,
    palette={"GPC" : "#DDDDDD", "VSV-G" : "#000000"},
)
chart.set_yscale("log")
chart.yaxis.set_minor_locator(mticker.NullLocator())  # no minor ticks
chart.set_ylim(1, 50000000)
yticks = [10, 1000, 100000, 10000000]
chart.set_yticks(yticks)
chart.set_yticklabels(labels=["$10^1$", "$10^3$", "$10^5$", "$10^7$"], fontsize=8)
xticks = [0, 1, 2, 3]
chart.set_xticks(xticks)
x_labels = [
    "293T", 
    "293T\u0394DAG1", 
    "293T\u0394DAG1+mastomysDAG1", 
    "293T\u0394DAG1+humanDAG1",
]
chart.set_xticklabels(labels=x_labels, rotation=90, horizontalalignment="center", fontsize=8)
chart.set_ylabel("TU/mL", fontsize=8)
chart.set(xlabel=None)
sns.move_legend(
    chart, 
    "upper left", 
    bbox_to_anchor=(1, 1),
    fontsize=8,
    markerscale=1,
    handletextpad=0.1,
    title="viral entry\nprotein",
    title_fontproperties = {
        "size" : 8, 
        "weight" : "bold",
    },
    frameon=False,
    borderaxespad=0.1,
)

# Change all spines
for axis in ["top", "bottom", "left", "right"]:
    chart.spines[axis].set_linewidth(1)
chart.tick_params(axis="both", length=4, width=1)

chart.grid(False)
sns.despine()

# Add edges to legend markers to match scatter plot
for ha in chart.legend_.legendHandles:
    ha.set_edgecolor("black")
    ha.set_linewidths(0.5)

# Make output dir if doesn't exist
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

# Save fig
plt.savefig(out_dir + "stable_cell_line_titers.svg")